# Implement the rotation of strain tensor

We check the computation of the rotation of strain tensor.

2023.5.18 Kurama Okubo

In [1]:
import datetime
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py
import glob

import shutil
from tqdm import tqdm

import matplotlib.patches as patches
import matplotlib as mpl

from BP_rotate_strain import *

%load_ext autoreload
%autoreload 2

os.environ['TZ'] = 'GMT' # change time zone to avoid confusion in unix_tvec conversion

plt.rcParams["font.family"] = 'Arial'
# plt.rcParams["font.sans-serif"] = "DejaVu Sans, Arial, Helvetica, Lucida Grande, Verdana, Geneva, Lucid, Avant Garde, sans-serif"
plt.rcParams["font.size"] = 12
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 5
plt.rcParams["xtick.major.width"] = 0.5
plt.rcParams["xtick.minor.size"] = 0
plt.rcParams["xtick.minor.width"] = 0
plt.rcParams["xtick.minor.visible"] = True


plt.rcParams["ytick.direction"] = "in"
plt.rcParams["ytick.major.size"] = 5
plt.rcParams["ytick.major.width"] = 0.5
plt.rcParams["ytick.minor.size"] = 2
plt.rcParams["ytick.minor.width"] = 1
plt.rcParams["ytick.minor.visible"] = True


In [2]:
output_imgdir = "./figure"
if not os.path.exists(output_imgdir):
    os.makedirs(output_imgdir)

# Theory

Let {$\varepsilon$} be 
$$
\begin{bmatrix}
\varepsilon_{ee} & \varepsilon_{en} \\
\varepsilon_{ne} & \varepsilon_{nn} \\
\end{bmatrix}
$$

in plane strain. The subscripts e and n indicate the east and north, respectively. Given the rotation to the fault is $\theta$, the fault-normal, fault-parallel and the shear strain along the fault is written as

$$ \varepsilon_{fn} = \varepsilon_{ee} \cos^2{\theta} + \varepsilon_{nn} \sin^2{\theta} + 2 \varepsilon_{en} \sin{\theta} \cos{\theta}  $$
$$ \varepsilon_{fp} = \varepsilon_{ee} \sin^2{\theta} + \varepsilon_{nn} \cos^2{\theta} - 2 \varepsilon_{en} \sin{\theta} \cos{\theta}  $$
$$ \varepsilon_{ss} = (\varepsilon_{nn}- \varepsilon_{ee}) \sin{\theta}\cos{\theta} + \varepsilon_{en}(\cos^2{\theta} - \sin^2{\theta}). $$

Then the new strain tensor is 
$$
\begin{bmatrix}
\varepsilon_{fn} & \varepsilon_{ss} \\
\varepsilon_{ss} & \varepsilon_{fp} \\
\end{bmatrix}
$$

Here we can check the invariant for the sake of debugging.

In [3]:
# Synthesize the stress
nu = 0.25
E = 100e9

# Check with onlign calculator

We cross-validate with the website below:
- https://www.efunda.com/formulae/solid_mechanics/mat_mechanics/calc_strain_transform.cfm

In [4]:
sxx0 = -20e6
syy0 = 10e6
tau0 = 6e6
theta_deg = 25
e1 =  compute_strain(sxx0, syy0, tau0, E, nu)

print(f"Eee={e1[0][0]:.8f}, Enn={e1[1][1]:.8f}, Een={e1[0][1]:.8f}")

Efn, Efp, Ess = rotate_strain(e1[0][0], e1[1][1], e1[0][1], theta_deg)
print(f"Efn={Efn}, Efp={Efp}, Ess={Ess}")

Eee=-0.00021875, Enn=0.00015625, Een=0.00007500
Efn=-9.431934358230276e-05, Efp=3.181934358230276e-05, Ess=0.0001918424038112988


In [5]:
# 1. answer
#   New Normal Strain ex':	 -9.43 × 10-5  	
#   New Normal Strain ey':	 3.18 × 10-5  	
#   New Shear Strain ex'y':	 1.92 × 10-4  
        

# Check with plots

In [6]:
sxx0 = -20e6
syy0 = 10e6
tau0 = 0 #10e6

e1 =  compute_strain(sxx0, syy0, tau0, E, nu)

In [7]:
theta_rotate = 45

Efn, Efp, Ess = rotate_strain(e1[0][0], e1[1][1], e1[0][1], theta_deg)
print(f"Efn={Efn}, Efp={Efp}, Ess={Ess}")

Efn=-0.0001517726768162261, Efp=8.92726768162261e-05, Ess=0.00014363333308480838


In [8]:
theta_list = np.linspace(0, 360, 13)

In [9]:

# theta_rotate = 45

for theta_rotate in theta_list:
    
    Efn, Efp, Ess = rotate_strain(e1[0][0], e1[1][1], e1[0][1], theta_rotate)
#     print(f"Efn={Efn}, Efp={Efp}, Ess={Ess}")

    # ref: https://stackoverflow.com/questions/4285103/matplotlib-rotating-a-patch
    fig, axs = plt.subplots(1, 3, figsize=(12, 4), sharex=False)

    norm = plt.Normalize(-10e-5, 10e-5)


    E_rotated = [Efn, Efp, Ess]
    titles = ["Eee->Efn", "Enn->Efp", "Een->Ess" ]
    for i, epsilon in enumerate([e1[0][0], e1[1][1], e1[0][1]]): # Eee, Enn, Een

        ce = plt.cm.coolwarm(norm(epsilon))
        cr = plt.cm.coolwarm(norm(E_rotated[i]))

        r1 = patches.Rectangle((-10,-10), 20, 20, edgecolor='k', alpha=0.5, facecolor=ce)
        r2 = patches.Rectangle((-5,-5), 10, 10, color=cr,  alpha=0.5, )

        t2 = mpl.transforms.Affine2D().rotate_deg(theta_rotate) + axs[i].transData
        r2.set_transform(t2)

        axs[i].add_patch(r1)
        axs[i].add_patch(r2)
        axs[i].set_title(titles[i])

        theta_rad = np.deg2rad(theta_rotate)
        L = 5
        if i==0:
            # axis 1
            axs[i].plot([0, L*np.cos(theta_rad)], [0, L*np.sin(theta_rad)], "y-")
        elif i==1:
            # axis 2
            axs[i].plot([0, L*np.cos(theta_rad+0.5*np.pi)], [0, L*np.sin(theta_rad+0.5*np.pi)], "y-")

    for i in range(3):
    #     axs[0].grid(True)
        axs[i].set_xlabel('EW')
        axs[i].set_ylabel('NS')
        axs[i].set_xlim([-20, 20])
        axs[i].set_ylim([-20, 20])
        axs[i].axis('equal')

    fig.tight_layout()
    
    fig.subplots_adjust(bottom=0.1)
    cbar_ax = fig.add_axes([0.0, -0.05, 1.0, 0.3])
    cbar_ax.set_axis_off()
    # (left, bottom, width, height)
    cbar=fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap="coolwarm"),
                     orientation='horizontal', ax=cbar_ax, location = 'bottom',
                     label="Strain")

    fig.suptitle('$\sigma ee$ ={:.2e} $\sigma nn$ ={:.2e}  $\sigma en$ ={:.2e}  '.format(sxx0, syy0, tau0), y=1.02)

    foname = output_imgdir+"/rotate_strain_rotate{:d}deg.png".format(int(theta_rotate))
    plt.savefig(foname, dpi=150, bbox_inches="tight")
    plt.close()
    plt.clf()

<Figure size 640x480 with 0 Axes>

In [10]:
e1[0][0]

-0.00021875